<a href="https://colab.research.google.com/github/CrystalWang1225/ECE-471-Frequentist-Machine-Learning/blob/main/ML7_Yingzhi%26_Crystal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install surprise

In [ ]:
import pandas as pd
import numpy as np
import surprise
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise.model_selection import GridSearchCV
from surprise.model_selection import train_test_split
from surprise import NMF

# Load Data
The dataset is from https://grouplens.org/datasets/movielens/
It contains informations on users ratings on different movies

In [ ]:
data = Dataset.load_builtin('ml-100k',prompt=False)

# Tuning Hyperparameter

In [ ]:
reg_all = [0.02,0.05,0.07]

param_grid = {
    "n_factors":[10,15,20],
    "n_epochs":[5,15,20],
    "reg_qi": reg_all,
    "reg_pu":reg_all
}

gs = GridSearchCV(NMF, param_grid,measures=["rmse","mae"],cv = 3)
gs.fit(data)

# best RMSE score
print("BEST RMSE: ")
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print("Best RMSE Parameter: ")
print(gs.best_params['rmse'])


BEST RMSE: 
0.9756106909609167
Best RMSE Parameter: 
{'n_factors': 15, 'n_epochs': 20, 'reg_qi': 0.07, 'reg_pu': 0.07}


In [ ]:
from surprise import accuracy
algo= gs.best_estimator['rmse']
trainset = data.build_full_trainset()
algo.fit(trainset)

testset = trainset.build_anti_testset()
prediction = algo.test(testset)
accuracy.rmse(prediction)

RMSE: 0.9080


0.9080405273981592

In [ ]:
# Load movie item dataset
columns_name = ["movie id","movie title","release date","video release date","IMDb URL","unknown","Action","Adventure","Animation","Children's","Comedy","Crime","Documentary","Drama","Fantasy","Film-Noir","Horror","Musical","Mystery","Romance","Sci-Fi","Thriller","War","Western"]
items = pd.read_csv("/root/.surprise_data/ml-100k/ml-100k/u.item",sep="|",names=columns_name,encoding='cp1250',header=0)
display = pd.DataFrame(columns=['user_id','movie title','rating','predicted_rating'])
count = 0

# Convert movie rating prediction list to a dataframe, replace the movie id with the movie name, and select the first 30000 prediction entries.
prediction_df = pd.DataFrame(columns=["user_id", "movie id", "movie title","predicted_rating"])
for each in prediction:
  each_item = items.loc[items['movie id'] == int(each.iid)]['movie title']
  prediction_df = prediction_df.append({"user_id": each[0], "movie id": each[1],"movie title":each_item.iloc[0] if len(each_item)==1 else "blank", "predicted_rating": each[3]},ignore_index=True)
  count = count + 1
  if count == 30000:
     break

# Randomize the prediction dataframe and select only one movie per user.
prediction_df = prediction_df.sample(frac=1)
prediction_df = prediction_df.drop_duplicates(subset=['user_id'], keep='last')

prediction_df.head(50)

,user_id,movie id,movie title,predicted_rating
21357,210,1329,"Low Life, The (1994)",2.061103
6563,166,210,Indiana Jones and the Last Crusade (1989),4.361299
13037,305,616,Night of the Living Dead (1968),3.332644
29842,119,1240,Ghost in the Shell (Kokaku kidotai) (1995),4.373600
26923,194,1173,Bliss (1997),2.670707
15275,6,1261,"Run of the Country, The (1995)",3.169294
28934,234,754,Red Corner (1997),3.025079
11290,253,612,Lost Horizon (1937),4.245377
7942,298,690,Seven Years in Tibet (1997),3.753729
1883,186,678,Volcano (1997),2.938476


Only the first 30000 rows in prediction is loaded into the dataframe. We randomized the rows, and only show one estimated ratings for each user.

# Stretch Goal 1